# OpenMC models
This notebook contains the code to create and run the OpenMC model of 
a PWR pincell, based on the [example online](https://github.com/openmc-dev/openmc/wiki/Example-Jupyter-Notebooks).
Running the model includes developing one-group cross section data and depletion using 
only UOX or MOX fuel.

In [6]:
import numpy as np
import openmc
import openmc.deplete as od

In [34]:
uox_model = openmc.Model()
mox_model = openmc.Model()
openmc.config['cross_sections'] = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"

In [35]:
# Materials
uo2 = openmc.Material(1, "uo2")
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)
uo2.volume = 0.602

zirconium = openmc.Material(name="zirconium")
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name="h2o")
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')

puo2 = openmc.Material(4)
puo2.add_nuclide('Pu239', 0.94)
puo2.add_nuclide('Pu240', 0.06)
puo2.add_nuclide('O16', 2.0)
puo2.set_density('g/cm3', 11.5)

# Create the mixture
mox = openmc.Material.mix_materials([uo2, puo2], [0.97, 0.03], 'wo')
mox.volume = 0.602

uox_model.Materials = openmc.Materials([uo2, zirconium, water])
mox_model.Materials = openmc.Materials([mox, zirconium, water])

uox_model.Materials.export_to_xml()

uox_model.materials.cross_sections = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"
mox_model.materials.cross_sections = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"

/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=4.
  warn(msg, IDWarning)


In [36]:
# Geometry
fuel_outer_radius = openmc.ZCylinder(r=0.39)
clad_inner_radius = openmc.ZCylinder(r=0.40)
clad_outer_radius = openmc.ZCylinder(r=0.46)

fuel_region = -fuel_outer_radius
gap_region = +fuel_outer_radius & -clad_inner_radius
clad_region = +clad_inner_radius & -clad_outer_radius

uox_fuel = openmc.Cell(name='fuel')
uox_fuel.fill = uo2
uox_fuel.region = fuel_region

mox_fuel = openmc.Cell(name='fuel')
mox_fuel.fill = mox
mox_fuel.region = fuel_region

gap = openmc.Cell(name='air gap')
gap.region = gap_region

clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

pitch = 1.26
box = openmc.rectangular_prism(width=pitch, height=pitch,
                               boundary_type='reflective')
water_region = box & +clad_outer_radius
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region

uox_universe = openmc.Universe(cells=(uox_fuel, gap, clad, moderator))
mox_universe = openmc.Universe(cells=(mox_fuel, gap, clad, moderator))

uox_model.geometry = openmc.Geometry(uox_universe)
uox_model.geometry.export_to_xml()
mox_model.geometry = openmc.Geometry(mox_universe)

In [37]:
# Create a point source
point = openmc.stats.Point((0, 0, 0))
source = openmc.Source(space=point)

/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/source.py:388: FutureWarning: This class is deprecated in favor of 'IndependentSource'
  warnings.warn("This class is deprecated in favor of 'IndependentSource'", FutureWarning)


In [38]:
# Settings
settings = openmc.Settings()
#settings.source = source
settings.batches = 100
settings.inactive = 10
settings.particles = 1000
settings.output = {'tallies':True}
uox_model.settings = settings
mox_model.settings = settings

In [39]:
# Generate one-group cross section data
uox_cross_sections = od.MicroXS.from_model(uox_model, 
                                       reaction_domain = uox_universe, 
                                       chain_file="chain_endfb71_pwr.xml")
mox_cross_sections = od.MicroXS.from_model(mox_model, 
                                       reaction_domain = mox_universe, 
                                       chain_file="chain_endfb71_pwr.xml")

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [14]:
# save cross sections from UOX model for use in Deplete Reactor
uox_cross_sections.to_csv("micro_xs.csv")

In [54]:
uox_cross_sections

,"(n,gamma)","(n,2n)","(n,p)","(n,a)","(n,3n)","(n,4n)",fission
U235,16.810290,3.828429e-03,1.818974e-07,4.161567e-05,6.280522e-06,0.0,22.551860
U238,4.171656,4.555887e-03,5.505988e-08,3.824461e-08,3.759061e-05,0.0,0.029991
O16,0.000057,1.916959e-09,8.681541e-06,3.411620e-03,0.000000e+00,0.0,0.000000
Zr90,0.008300,5.022449e-05,9.397042e-05,6.427602e-06,0.000000e+00,0.0,0.000000
Zr91,0.304000,7.762554e-04,5.493404e-05,1.391245e-04,0.000000e+00,0.0,0.000000
Zr92,0.052334,4.871039e-04,6.775038e-06,1.372729e-05,3.215425e-07,0.0,0.000000
Zr94,0.019562,7.792530e-04,8.188245e-07,1.031367e-06,1.555524e-06,0.0,0.000000
Zr96,0.135538,1.089385e-03,1.044709e-07,1.384115e-07,3.512083e-06,0.0,0.000000
H1,0.048562,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000


In [55]:
mox_cross_sections

,"(n,gamma)","(n,2n)","(n,p)","(n,a)","(n,3n)","(n,4n)",fission
U235,7.657629,0.005067,1.673230e-07,1.489132e-05,0.000007,0.0,30.420866
U238,0.841209,0.006069,4.062161e-08,1.368654e-08,0.000056,0.0,0.118806
O16,0.000052,0.000000,9.082989e-06,4.438850e-03,0.000000,0.0,0.000000
Pu239,27.757456,0.001549,0.000000e+00,0.000000e+00,0.000004,0.0,54.511987
Pu240,112.326662,0.001320,0.000000e+00,0.000000e+00,0.000005,0.0,0.644986


In [40]:
uox_ind_op = od.IndependentOperator(uox_model.Materials, 
                                uox_cross_sections, 
                                "chain_endfb71_pwr.xml")
mox_ind_op = od.IndependentOperator(mox_model.Materials, 
                                mox_cross_sections, 
                                "chain_endfb71_pwr.xml")

In [41]:
uox_integrator = od.PredictorIntegrator(uox_ind_op,
                                    np.ones(12*3)*30, # 3 cycles of operation
                                   power = 278.7,
                                   timestep_units='d')
uox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=33696000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=36288000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=38880000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t

In [43]:
! mv depletion_results.h5 uox_depletion_results.h5

In [59]:
mox_integrator = od.PredictorIntegrator(mox_ind_op,
                                    np.ones(12*3)*30,
                                   power = 278.7,
                                   timestep_units='d')
mox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=33696000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=36288000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t=38880000.0 s, dt=2592000.0 s, source=278.7
[openmc.deplete] t

In [60]:
! mv depletion_results.h5 mox_depletion_results.h5

In [44]:
# Print depletion material compositions
uox_results = od.Results("uox_depletion_results.h5")
mox_results = od.Results("mox_depletion_results.h5")

In [45]:
spent_uox = uox_results[-1].get_material("1")
spent_mox = mox_results[-1].get_material("9")

In [52]:
uox_results.export_to_materials(1)

[Material
 	ID             =	1
 	Name           =	uo2
 	Temperature    =	None
 	Density        =	10.0 [sum]
 	Volume         =	0.602 [cm^3]
 	S(a,b) Tables  
 	Nuclides       
 	Ac225          =	8.651786098729872e-25 [ao]
 	Ac227          =	4.1164375212067685e-20 [ao]
 	Ag107          =	3.293607788763919e-16 [ao]
 	Ag109          =	1.455048764523584e-08 [ao]
 	Ag110_m1       =	7.729052989789892e-19 [ao]
 	Ag111          =	2.387839406106799e-09 [ao]
 	Ag112          =	3.31717090667822e-11 [ao]
 	Ag113          =	3.888995052104574e-11 [ao]
 	Ag114          =	1.178230895406513e-14 [ao]
 	Ag115          =	2.981101684119229e-12 [ao]
 	Ag116          =	6.369795901812032e-13 [ao]
 	Ag117          =	1.7133433143267955e-13 [ao]
 	Ag118_m1       =	1.4577356741398455e-15 [ao]
 	As74           =	3.4393185493084262e-18 [ao]
 	As75           =	3.8334580857241166e-10 [ao]
 	Ba132          =	3.4524748249718125e-16 [ao]
 	Ba133          =	3.6667157749285695e-19 [ao]
 	Ba134          =	3.793625028139904

In [53]:
uox_results.export_to_materials(0)

[Material
 	ID             =	1
 	Name           =	uo2
 	Temperature    =	None
 	Density        =	10.0 [sum]
 	Volume         =	0.602 [cm^3]
 	S(a,b) Tables  
 	Nuclides       
 	O16            =	0.044616644596722044 [ao]
 	U235           =	0.0006692496689508309 [ao]
 	U238           =	0.021639072629410193 [ao],
 Material
 	ID             =	4
 	Name           =	zirconium
 	Temperature    =	None
 	Density        =	6.6 [g/cm3]
 	Volume         =	None [cm^3]
 	S(a,b) Tables  
 	Nuclides       
 	Zr90           =	0.5145       [ao]
 	Zr91           =	0.1122       [ao]
 	Zr92           =	0.1715       [ao]
 	Zr94           =	0.1738       [ao]
 	Zr96           =	0.028        [ao],
 Material
 	ID             =	5
 	Name           =	h2o
 	Temperature    =	None
 	Density        =	1.0 [g/cm3]
 	Volume         =	None [cm^3]
 	S(a,b) Tables  
 	S(a,b)         =	('c_H_in_H2O', 1.0)
 	Nuclides       
 	H1             =	2.0          [ao]
 	O16            =	1.0          [ao]]

In [47]:
#printing in format for Cyclus recipe
total = 0
weight_frac = 0
for nuclide in spent_uox.nuclides:
    total += nuclide.percent
    if nuclide.percent <= 1e-15:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent/100}</comp>  </nuclide>")
    weight_frac += nuclide.percent*A
    total += nuclide.percent

<nuclide>  <id>20040000</id> <comp>0.0004353093228652388</comp>  </nuclide>
<nuclide>  <id>60120000</id> <comp>1.340167958242337e-11</comp>  </nuclide>
<nuclide>  <id>60130000</id> <comp>0.00043887600532518923</comp>  </nuclide>
<nuclide>  <id>70150000</id> <comp>2.4660055129028867e-10</comp>  </nuclide>
<nuclide>  <id>80170000</id> <comp>7.290180639807649e-06</comp>  </nuclide>
<nuclide>  <id>290650000</id> <comp>3.449844976888058e-16</comp>  </nuclide>
<nuclide>  <id>300660000</id> <comp>5.959541137561211e-14</comp>  </nuclide>
<nuclide>  <id>300670000</id> <comp>3.4473948804444974e-13</comp>  </nuclide>
<nuclide>  <id>300680000</id> <comp>4.597885963583949e-13</comp>  </nuclide>
<nuclide>  <id>300700000</id> <comp>8.227485664919575e-13</comp>  </nuclide>
<nuclide>  <id>310690000</id> <comp>6.144802728862363e-13</comp>  </nuclide>
<nuclide>  <id>310710000</id> <comp>1.0849716619616972e-12</comp>  </nuclide>
<nuclide>  <id>320720000</id> <comp>1.9578405016303556e-12</comp>  </nuclide>

In [48]:
pu_weight_frac = (0.0214*239)/weight_frac
pu_weight_frac

0.04736567610367108

In [49]:
total

2.3229331792434427

In [74]:
#printing in format for Cyclus recipe
for nuclide in spent_mox.nuclides:
    if nuclide.percent <= 1e-15:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent/100}</comp>  </nuclide>")

<nuclide>  <id>10010000</id> <comp>1.8770776671948985e-10</comp>  </nuclide>
<nuclide>  <id>20040000</id> <comp>9.25971362362798e-08</comp>  </nuclide>
<nuclide>  <id>60120000</id> <comp>2.2524927949491653e-15</comp>  </nuclide>
<nuclide>  <id>60130000</id> <comp>9.173263529542912e-08</comp>  </nuclide>
<nuclide>  <id>70160000</id> <comp>3.380705933986699e-17</comp>  </nuclide>
<nuclide>  <id>80160000</id> <comp>0.000447769281791141</comp>  </nuclide>
<nuclide>  <id>80170000</id> <comp>1.074047009425417e-09</comp>  </nuclide>
<nuclide>  <id>280660000</id> <comp>2.6752242057325917e-16</comp>  </nuclide>
<nuclide>  <id>290650000</id> <comp>2.570072254662584e-16</comp>  </nuclide>
<nuclide>  <id>290670000</id> <comp>1.503473678103865e-15</comp>  </nuclide>
<nuclide>  <id>300660000</id> <comp>6.216129441963165e-14</comp>  </nuclide>
<nuclide>  <id>300670000</id> <comp>2.9235770762626664e-13</comp>  </nuclide>
<nuclide>  <id>300680000</id> <comp>4.773460208998419e-13</comp>  </nuclide>
<nuc

In [56]:
results = od.Results("/home/abachmann@anl.gov/openmcyclus/comparison/depletion_results.h5")

In [57]:
spent_comps = []

In [ ]:
for index, material_id in 